In [1]:
import pandas as pd
import sys
import os
sys.path.append("/Users/marc/Desktop/PostDoc/Mutation/signature5/notebooks")
import tools

col_order = ['Chr', 'Pos', 'Ref', 'Alt', 'Sample', 'Tissue_CellType', 'Compartment',
             'Individual', 'Study', 'Context', 'GeneralType', 'Type']
col_meta = ["Study", "Sample", "Individual", "Age", "Compartment", "Status", "Tissue_CellType"]

Read data

In [1]:
dat = pd.read_excel("1-s2.0-S0092867424002277-mmc3.xlsx")
dat = dat.rename(columns = {"refnt":"Ref", "altnt":"Alt", "sample":"Sample", "subject":"Individual"})

In [2]:
dat

,Sample,chr,pos,Ref,Alt,muttype,mutsig,pass,rescue,Individual,id,rec.filter,nearest,cluster.filter,lineage.filter,final.filter,Unnamed: 16,Unnamed: 17
0,5657_OL2,1,568588,G,A,snv,ACG:C>T,True,False,UMB5657,1 568588 G A,False,1444496.0,False,False,False,NaN,UMBUMB5657
1,5817_OL2,1,783125,G,A,snv,ACT:C>T,True,False,UMB5817,1 783125 G A,False,4778488.0,False,False,False,NaN,UMBUMB5817
2,936-Oligo-6,1,847408,G,A,snv,GCT:C>T,True,False,UMB936,1 847408 G A,False,218760.0,False,False,False,NaN,UMBUMB936
3,5657PFC-B,1,853262,C,G,snv,GCC:C>G,True,False,UMB5657,1 853262 C G,False,15129252.0,False,False,False,NaN,UMBUMB5657
4,4643_OL3,1,918639,C,T,snv,CCG:C>T,True,False,UMB4643,1 918639 C T,False,4057259.0,False,False,False,NaN,UMBUMB4643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93776,GliaLC-4-H8,22,50870152,G,A,snv,CCG:C>T,True,False,UMB5657,22 50870152 G A,False,632.0,False,False,False,NaN,UMBUMB5657
93777,5657-Oligo-3,22,51060461,T,C,snv,TTA:T>C,False,True,UMB5657,22 51060461 T C,False,1102927.0,False,False,False,NaN,UMBUMB5657
93778,5823_OL4,22,51126380,C,T,snv,GCG:C>T,True,False,UMB5823,22 51126380 C T,False,6299881.0,False,False,False,NaN,UMBUMB5823
93779,GliaLC-6-B10,22,51138847,T,A,snv,CTG:T>A,True,False,UMB5871,22 51138847 T A,False,8045981.0,False,False,False,NaN,UMBUMB5871


In [2]:
meta1 = pd.read_excel("1-s2.0-S0092867424002277-mmc2.xlsx")
meta2 = pd.read_excel("1-s2.0-S0092867424002277-mmc1.xlsx", skiprows=1)

In [3]:
cell2type = meta1.set_index("sample")["type"].to_dict()
cell2type = {k:v.capitalize() for k,v in cell2type.items()}

In [4]:
dat["Tissue_CellType"] = dat.Sample.map(cell2type)
dat["Study"] = "Ganz2024"
dat["Compartment"] = "WGS"
dat["Chr"] = "chr" + dat["chr"].astype("str")
dat["Pos"] = dat.pos.astype(int)

In [57]:
dat = tools.keep_snps(dat).reset_index(drop="True")
dat = dat[~dat["Chr"].isin(["chrX", "chrY"])]

In [58]:
cosmic = tools.COSMIC()
dat = tools.annotate_context(dat, "../fasta/hg19.fa")
dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
dat['Type'] = cosmic.fold(dat['GeneralType'])

In [66]:
dat = dat[col_order]
dat.to_csv("Ganz2024.csv.gz", compression="gzip", index=False)

Metafile

In [7]:
meta2

,Life stage,Donor ID,Age,Sex,Neurons,OLs,OLs.1,Mixed glia,Unnamed: 8,Unnamed: 9,Neurons.1,OLs.2
0,Infant,UMB1278,0.4,M,3.0,6.0,5.0,5.0,NaN,NaN,NaN,NaN
1,NaN,UMB5817,0.6,M,3.0,6.0,NaN,NaN,NaN,SNVs,359.0,1320.0
2,NaN,UMB5871,2,M,3.0,6.0,5.0,5.0,NaN,Indels,21.0,42.0
3,NaN,NaN,NaN,NaN,9.0,18.0,10.0,10.0,NaN,NaN,NaN,NaN
4,Adolescent,UMB4638,15.1,F,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,UMB1465,17.5,M,4.0,6.0,NaN,NaN,NaN,SNVs,1457.0,3967.0
6,NaN,UMB5559,19.8,F,3.0,5.0,NaN,NaN,NaN,Indels,222.0,202.0
7,NaN,NaN,NaN,NaN,10.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN
8,Adult,UMB4643,42.2,F,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,UMB5087,44.9,M,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
meta1

,donor,sample,amp,type,selection,age,ageclass,batch,outlier,sex,plotage,color
0,UMB1278,GliaLC-5-B12,MDA,oligo,SOX10,0.4,infant,1,NORMAL,M,0.0,orange
1,UMB1278,GliaLC-5-B7,MDA,mixed,GFAP,0.4,infant,1,NORMAL,M,0.0,purple
2,UMB1278,GliaLC-5-B8,MDA,mixed,GFAP,0.4,infant,1,NORMAL,M,0.0,purple
3,UMB1278,GliaLC-5-C7,MDA,mixed,GFAP,0.4,infant,1,NORMAL,M,0.0,purple
4,UMB1278,GliaLC-5-D10,MDA,oligo,SOX10,0.4,infant,1,NORMAL,M,0.0,orange
...,...,...,...,...,...,...,...,...,...,...,...,...
178,UMB5817,5817_liver_bulk,bulk,bulk,bulk,0.6,infant,1,NORMAL,M,0.6,green
179,UMB5823,5823-tempmusc-1b1_20170221-WGS,bulk,bulk,bulk,82.7,elderly,1,NORMAL,F,82.7,green
180,UMB936,936-hrt-1b1_20170221-WGS,bulk,bulk,bulk,49.2,adult,1,NORMAL,F,49.2,green
181,UMB5559,5559-bulk,bulk,bulk,bulk,19.8,adolescent,1,NORMAL,F,19.8,green


In [8]:
meta = meta1.rename(columns = {"sample":"Sample", "donor":"Individual","age":"Age", "type":"Tissue_CellType", "amp":"Sequencing"})

In [9]:
meta["Study"] = "Ganz2024"
meta["Status"] = "Normal"
meta["Compartment"] = "WGS"

In [10]:
meta[col_meta + ["Sequencing"]].to_csv("Ganz2024.tab.gz",sep="\t", index=False)

In [15]:
!gunzip -c Ganz2024.tab.gz 

Study	Sample	Individual	Age	Compartment	Status	Tissue_CellType	Sequencing
Ganz2024	GliaLC-5-B12	UMB1278	0.4	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-5-B7	UMB1278	0.4	WGS	Normal	mixed	MDA
Ganz2024	GliaLC-5-B8	UMB1278	0.4	WGS	Normal	mixed	MDA
Ganz2024	GliaLC-5-C7	UMB1278	0.4	WGS	Normal	mixed	MDA
Ganz2024	GliaLC-5-D10	UMB1278	0.4	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-5-E12	UMB1278	0.4	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-5-E7	UMB1278	0.4	WGS	Normal	mixed	MDA
Ganz2024	GliaLC-5-G7	UMB1278	0.4	WGS	Normal	mixed	MDA
Ganz2024	GliaLC-5-H10	UMB1278	0.4	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-5-H11	UMB1278	0.4	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-6-A5	UMB5171	79.0	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-6-B4	UMB5171	79.0	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-6-B5	UMB5171	79.0	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-6-C1	UMB5171	79.0	WGS	Normal	mixed	MDA
Ganz2024	GliaLC-6-C2	UMB5171	79.0	WGS	Normal	mixed	MDA
Ganz2024	GliaLC-6-E6	UMB5171	79.0	WGS	Normal	oligo	MDA
Ganz2024	GliaLC-6-F3	UMB5171	79.0	WGS	Normal	mixed	